In [22]:
import json
import pandas as pd
import math
from numpy import diff
from numpy import gradient
import numpy as np
from scipy import pi
from scipy import fft
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

def lowpass_filter(column,dict):#column, cutoff, order, sample_rate):
    cutoff = dict['cutoff']
    order = dict['order']  
    rate = dict['rate']
    b,a = signal.butter(order,cutoff/(rate/2),btype='lowpass')
    filtered_channel = signal.filtfilt(b,a,column)
    return (filtered_channel)

def get_psd(df,fs,bin_width):
    
    #fs = len(df)/(df.index[-1]-df.index[0])    
    f, psd = signal.welch(df.to_numpy(), 
                          fs=fs, 
                          nperseg=fs/bin_width,
                          #window='hanning',
                          axis=0
                         )

    df_psd = pd.DataFrame(psd)#,columns=df.columns)
    df_psd.columns
    df_psd['Frequency (Hz)'] = f
    df_psd = df_psd.set_index('Frequency (Hz)')
    
    return df_psd[1:] #drop the first value because it makes the plots look bad and is effectively 0

# gps_data = data['locations']  # extra GPS entry
# frame=pd.DataFrame(gps_data) # Convert to data frame

# frame['elapsed_time'] = frame['tUTC'] - frame['tUTC'][0] # change time values to relative for this run

# susp=data['suspension']
# df=pd.DataFrame(susp)  # choose suspension entry and create dataframe
# df=df[:snip]


#clip zero values/sensor dropouts & generate velocity columns
def process_sensor_data(df,rate):
    size=len(df.index)
    timedata,interval=np.linspace(0,size/rate,num=size,retstep=True, endpoint=False) #create timestamps
    df['time']=np.round(timedata,3) # add to dataframe, round to 0.001s
    df.where(df.fork>0,inplace=True)
    df.where(df.shock>0,inplace=True)
    df.where(df.rearWheel>0,inplace=True)
    x=df.min()
    df.fork=df.fork-x['fork']
    df.shock=df.shock-x['shock']
    df.rearWheel=df.rearWheel-x['rearWheel']
    df['forkVelocity']=np.gradient(df.fork,df.time)
    df['shockVelocity']=np.gradient(df.shock,df.time)
    df['rearVelocity']=np.gradient(df.rearWheel,df.time)
    df.dropna(how='any', inplace=True)
    return df

# rider_data = {
#     'forktravel' : 160,
#     'wheeltravel' : 146,
#     'shocktravel' : 60,
# }    

In [21]:

df=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-04-16-02/OPPOFindX5Pro-04-16-02.csv',usecols=[1,2,3,4,5], 
               names=( 'fork','rearWheel','frontForce','shock','rearForce'), header=0)
df

,fork,rearWheel,frontForce,shock,rearForce
0,97.59,56.60,-11.69,21.41,-2.32
1,103.50,57.47,-11.69,21.77,-2.32
2,103.84,58.52,-5.95,22.19,-0.63
3,101.69,59.16,-5.95,22.45,-0.63
4,98.56,59.67,3.83,22.66,0.67
...,...,...,...,...,...
46380,44.71,-9.99,-1.14,-9.99,0.00
46381,44.76,-9.99,-1.14,-9.99,0.00
46382,44.76,-9.99,-1.13,-9.99,0.00
46383,44.71,-9.99,-1.13,-9.99,0.00


In [23]:
df2=process_sensor_data(df,200)
df2

,fork,rearWheel,frontForce,shock,rearForce,time,forkVelocity,shockVelocity,rearVelocity
0,73.39,52.13,-11.69,19.85,-2.32,0.000,1182.0,72.0,174.0
1,79.30,53.00,-11.69,20.21,-2.32,0.005,625.0,78.0,192.0
2,79.64,54.05,-5.95,20.63,-0.63,0.010,-181.0,68.0,169.0
3,77.49,54.69,-5.95,20.89,-0.63,0.015,-528.0,47.0,115.0
4,74.36,55.20,3.83,21.10,0.67,0.020,-606.0,23.0,55.0
...,...,...,...,...,...,...,...,...,...
46366,20.51,34.64,-1.13,12.89,0.55,231.830,5.0,-1.0,-5.0
46367,20.56,34.59,-1.13,12.88,0.55,231.835,5.0,-1.0,-5.0
46368,20.56,34.59,-1.13,12.88,0.56,231.840,-5.0,0.0,0.0
46369,20.51,34.59,-1.13,12.88,0.56,231.845,-5.0,1.0,5.0


In [24]:
df2.describe()

,fork,rearWheel,frontForce,shock,rearForce,time,forkVelocity,shockVelocity,rearVelocity
count,46371.000000,46371.000000,46371.000000,46371.000000,46371.000000,46371.000000,46371.000000,46371.000000,46371.000000
mean,41.903142,38.791211,-1.053213,14.697550,0.385417,115.925000,-0.215221,-0.029264,-0.073667
std,22.440007,19.996564,2.274537,7.957487,1.671010,66.931495,472.651462,144.544044,364.104668
min,0.000000,0.000000,-16.000000,0.000000,-12.650000,0.000000,-1763.000000,-463.000000,-1064.000000
25%,25.780000,25.545000,-2.000000,9.390000,-0.420000,57.962500,-201.000000,-65.000000,-165.000000
50%,39.600000,36.400000,-0.940000,13.580000,0.340000,115.925000,-10.000000,-8.000000,-20.000000
75%,57.270000,49.920000,0.240000,18.960000,1.000000,173.887500,63.000000,23.000000,58.000000
max,130.660000,131.510000,10.320000,53.790000,14.190000,231.850000,4468.000000,2505.000000,6090.000000


In [25]:
import plotly.figure_factory as ff
a=df2['forkVelocity']
b=df2['rearVelocity']
hist_data = [a,b]
group_labels = ['fork','rear wheel'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels,show_rug=False, bin_size=50
                         )
# fig.update_yaxes(range=[0,0.05])
fig.update_xaxes(range=[-1000,1000])
fig.show()